In [1]:
import os
import gc
import torch
import torch.nn as nn
import numpy as np
import nibabel as nib
import statsmodels.api as sm                                                                                        
from os.path import join as pjoin
from sklearn.linear_model import LinearRegression
from scipy import stats
from scipy.stats import zscore
from joblib import Parallel, delayed
import time
from utils import train_data_normalization, Timer, net_size_info, conv2_labels

In [2]:
sub = 'sub-01'
inputlayername = 'googlenet-conv2' 
layer = {'name': inputlayername, 'size': net_size_info[inputlayername.replace('raw-', '')]}
layername = layer['name']
layername = layername.replace('.','')
labels = conv2_labels
mask_name = 'primaryvis-in-MMP' #'fixretfloc-in-subj'
test_set_name = 'coco'
print(sub, mask_name, layername)
fold_indices = [(0, 1000), (1000, 2000), (2000, 3000), (3000, 4000)]
# path settings
work_dir = '/nfs/z1/userhome/GongZhengXin/NVP/NaturalObject/data/code/nodretinotopy/mfm_locwise_fullpipeline/'
# input path
resp_path = pjoin(work_dir, 'prep/brain_response')
voxel_mask_path = pjoin(work_dir, 'prep/voxel_masks')
image_activations_path = pjoin(work_dir, 'prep/image_activations')
retino_path = pjoin(work_dir, 'build/retinoparams')
guass_path = pjoin(work_dir, 'build/gaussianparams')
performance_path = pjoin(work_dir, 'build/featurewise-corr/conv1-pca-nonlinear')
# save path

sub-01 primaryvis-in-MMP googlenet-conv2
